## Desarrollo:
    - Explicar los métodos numéricos (CMM,WP,?)  
    - Se deben mencionar los pasos que siguieron
para implementar los algoritmos, las dificultades que fueron encontrando y la descripci ́on de c ́omo las fueron resolviendo.  
-incluir explicacion de por que el algoritmo funciona correctamente utilizando CMM  
-alternativas consideradas y descartadas para cada uno de los métodos utilizados (CMM, WP, Keener)   


### Características estudiadas:  
1. C es estrictamente diagonal dominante  
2. Al ser una matriz estrictamente diagonal dominante, esta es inversible y tiene una única solución. Como tiene una única solución, no será necesario hacer ningún intercambio de filas o columnas durante la EG ya que no tendremos ecuaciones sin solución o infinitas soluciones.  
3. ?  
